## Importing the relevant packages

In [8]:
import os
import sys
os.getcwd() 
sys.path.insert(0, "/Users/junxiong/Documents/HOO/MFTreeSearchCV-master/")
# sys.path.append("/Users/junxiong/Documents/HOO/MFTreeSearchCV-master/")

In [16]:
!pip install xgboost

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 665 kB 6.5 MB/s eta 0:00:01
  Created wheel for xgboost: filename=xgboost-0.82-cp27-cp27m-macosx_10_15_x86_64.whl size=1677831 sha256=7959785f983270cdd6a59e549f5286c8333885da70342302e79b95b7a9530120
  Stored in directory: /Users/junxiong/Library/Caches/pip/wheels/ac/4b/14/5a9e6da5b114f3d203ee2488576dfbe905193fcbee83e9fa4a
Successfully built xgboost


In [18]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
from sklearn.svm import SVC,SVR
import os
import sys
from MFTreeSearchCV.MFTreeSearchCV import *
from mf.mf_func import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
from sklearn.linear_model import LogisticRegression

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Just listing the contents of the main code directory

In [12]:
%ls MFTreeSearchCV/

MFHOO.py            MFTreeSearchCV.py   __init__.pyc        converters.pyc
MFTreeFunction.py   MFTreeSearchCV.pyc  __pycache__/
MFTreeFunction.pyc  __init__.py         converters.py


## Fetching some common data-sets 
- the news group dataset will be used in this example

In [19]:
from sklearn.datasets import load_digits,load_boston,fetch_20newsgroups
data = load_boston()

In [20]:
newsgroups_train = fetch_20newsgroups(subset='all')
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(newsgroups_train.data)
labels = newsgroups_train.target
#features =features.todense()

## Creating the features X and the target y
- Note that there are 15076 samples in the train set
- This will be the number of samples to be used a the highest fidelity in the next cell

In [21]:
X = features
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape

(15076, 173762)

## Setting up the estimator and parameters
- As a small example we will simple tune logistic regression
- param_dict spcifies that we will be tuning 'C' and 'penalty'
- 'C' is a real valued parameter to be tuned in the range [1e-5,1e5] and the searching is done in the log scale
- 'penalty' is of course a categorical parameter
- n_jobs is the number of threads used during CV
- cv = 3 implies 3-fold cross-validation
- scoring is set as 'accuracy'
- fidelity range is [500,15076], that is at the highest level we caan traain on the whole data-set while at the cheapest level we can train using only 500 samples chosen at random

In [22]:
estimator = LogisticRegression() #base estimator
param_dict = {'C':{'range':[1e-5,1e5],'scale':'log','type':'real'},\
              'penalty':{'range':['l1','l2'],'scale':'linear','type':'cat'}} #parameter space
fidelity_range = [500,15076] # fidelity range, lowest fidelity uses 500 samples while the highest one uses 
#the whole dataset  
n_jobs = 3 # number of jobs
cv = 3 # cv level
fixed_params = {}
scoring = 'accuracy'

## Budget
- We set the total budget as 100 secs
- This is may be only 3-4 times the budget required to do one single training and CV using the whole data-set

In [23]:
t1 = time.time()
estimator = estimator.fit(X_train,y_train)
t2 = time.time()
total_budget = 100 # total budget in seconds
print('Time without CV: ', t2 - t1)

/Users/junxiong/Library/Python/2.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/junxiong/Library/Python/2.7/lib/python/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


('Time without CV: ', 9.831360101699829)


## Creating an instance of the class

In [35]:
model = MFTreeSearchCV(estimator=estimator,param_dict=param_dict,scoring=scoring,\
                      fidelity_range=fidelity_range,unit_cost=None,\
                    cv=cv,  n_jobs = n_jobs,total_budget=total_budget,debug = True,fixed_params=fixed_params)

## running in debug mode will display certain outputs

## Fitting the model or choosing the best parameter
- Note that refit = true, which means at the end the training is done at the best parameter

In [37]:
m = model.fit(X_train,y_train)

Setting unit cost automatically as None was supplied
Unit Cost:  8.09670615196
Auto Init: 
C: 0.20886576371957724
nu: 0.20886576371957724
Budget Remaining: 88.2185938009
Number of MFHOO Instances: 3
Budget per MFHOO Instance:21.3094917817
Running SOO number: 1 rho: 0.95 nu: 0.20886576371957724
multi-thread search here
finish 2222
finish 1
multi-thread search here
finish 2222
finish 1
multi-thread search here
finish 2222
finish 1
Done!
Running SOO number: 2 rho: 0.925945462757 nu: 0.20886576371957724
multi-thread search here
finish 2222
finish 1
multi-thread search here
finish 2222
finish 1
Done!
Updating C
C: 0.29241206920740814
nu_max: 0.29241206920740814
Running SOO number: 3 rho: 0.857375 nu: 0.29241206920740814
multi-thread search here
finish 2222
finish 1
multi-thread search here
finish 2222
finish 1
Done!
Updating C
C: 0.4093768968903714
nu_max: 0.4093768968903714


## Predicting using the best model, then scoring it and then displaying the best_params_

In [26]:
y_pred = m.predict(X_test)

In [27]:
accuracy_score(y_pred,y_test)

0.926525198938992

In [28]:
m.best_params_

{'C': 316.2277660168377, 'penalty': 'l2'}

In [29]:
m.cv_results_

,params,score
0,"{u'penalty': u'l2', u'C': 5623.413251903499}",0.948063
1,"{u'penalty': u'l2', u'C': 5623.413251903499}",0.951514
2,"{u'penalty': u'l2', u'C': 316.2277660168377}",0.952177
